In [1]:
import sys
sys.path.append('../smodels')
from smodels.tools import runtime
#Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
#runtime.modelFile = 'mssmQNumbers.slha'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
setLogLevel("info")

In [2]:
# Set the path to the database
database = Database("/home/lessa/.cache/smodels/official200.pcl")

INFO in databaseObj.loadBinaryFile() in 483: loading binary db file /home/lessa/.cache/smodels/official200.pcl format version 213
INFO in databaseObj.loadBinaryFile() in 490: Loaded database from /home/lessa/.cache/smodels/official200.pcl in 2.0 secs.


In [3]:
model = Model(BSMparticles=BSMList, SMparticles=SMList)
slhafile = '../EWino/data/slha_scanRandom/ew_8hc5sqqr.slha'
model.updateParticles(inputFile=slhafile)


# Set main options for decomposition
sigmacut = 0.005*fb
mingap = 5.*GeV

# Decompose model
toplist = decomposer.decompose(model, sigmacut, doCompress=True, doInvisible=True, minmassgap=mingap)


INFO in model.updateParticles() in 385: Loaded 62 BSM particles


In [4]:
elList = sorted(toplist.getElements(), key = lambda el:el.weight.getMaxXsec(), reverse=True)
for el in elList:
    print(el,el.oddParticles,el.weight.getMaxXsec())
    print(el.mass,'\n')

[[],[]] [[N1], [C1+]] 1.21E+00 [pb]
[[1.68E+02 [GeV]], [1.84E+02 [GeV]]] 

[[],[]] [[C1-], [C1+]] 7.98E-01 [pb]
[[1.84E+02 [GeV]], [1.84E+02 [GeV]]] 

[[],[]] [[C1-], [N1]] 6.73E-01 [pb]
[[1.84E+02 [GeV]], [1.68E+02 [GeV]]] 

[[],[[q,q]]] [[N1], [C1+, N1]] 4.05E-01 [pb]
[[1.68E+02 [GeV]], [1.84E+02 [GeV], 1.68E+02 [GeV]]] 

[[],[[c,q]]] [[N1], [C1+, N1]] 4.03E-01 [pb]
[[1.68E+02 [GeV]], [1.84E+02 [GeV], 1.68E+02 [GeV]]] 

[[],[[q,q]]] [[C1-], [C1+, N1]] 2.68E-01 [pb]
[[1.84E+02 [GeV]], [1.84E+02 [GeV], 1.68E+02 [GeV]]] 

[[],[[q,q]]] [[C1+], [C1-, N1~]] 2.68E-01 [pb]
[[1.84E+02 [GeV]], [1.84E+02 [GeV], 1.68E+02 [GeV]]] 

[[],[[c,q]]] [[C1-], [C1+, N1]] 2.66E-01 [pb]
[[1.84E+02 [GeV]], [1.84E+02 [GeV], 1.68E+02 [GeV]]] 

[[],[[c,q]]] [[C1+], [C1-, N1~]] 2.66E-01 [pb]
[[1.84E+02 [GeV]], [1.84E+02 [GeV], 1.68E+02 [GeV]]] 

[[],[[q,q]]] [[N1], [C1-, N1~]] 2.26E-01 [pb]
[[1.68E+02 [GeV]], [1.84E+02 [GeV], 1.68E+02 [GeV]]] 

[[],[[c,q]]] [[N1], [C1-, N1~]] 2.25E-01 [pb]
[[1.68E+02 [GeV]], [1

In [5]:
el.weight.getMaxXsec()

5.38E-06 [pb]

In [11]:
exp = database.getExpResults(analysisIDs=['CMS-SUS-17-004'],txnames='TChiWZoff')[0]

In [12]:
tx = exp.getTxNames()[0]
print(tx)

TChiWZoff


In [8]:
tx.constraint

'71.*([[[mu+,mu-]],[[l,nu]]] + [[[e+,e-]],[[l,nu]]])'

In [13]:
tx.txnameData.getValueFor([[184*GeV,167*GeV]]*2)

2.67E+03 [fb]

In [10]:
# Compute the theory predictions for each experimental result and print them:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
listOfExpRes = database.getExpResults()
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, toplist, combinedResults=False, marginalize=False)
    if not predictions: continue # Skip if there are no constraints from this result
    print('\n %s ' %expResult.globalInfo.id)
    for theoryPrediction in predictions:
        dataset = theoryPrediction.dataset
        datasetID = theoryPrediction.dataId()
        mass = theoryPrediction.mass
        txnames = [str(txname) for txname in theoryPrediction.txnames]
        PIDs =  theoryPrediction.PIDs         
        print("------------------------" )
        print("Dataset = ",datasetID )   #Analysis name
        print("TxNames = ",txnames )  
        print("Prediction Mass = ",mass )   #Value for average cluster mass (average mass of the elements in cluster)
        print("Prediction PIDs = ",PIDs )   #Value for average cluster mass (average mass of the elements in cluster)
        print("Theory Prediction = ",theoryPrediction.xsection )  #Signal cross section
        print("Condition Violation = ",theoryPrediction.conditions ) #Condition violation values

        # Get the corresponding upper limit:
        print("UL for theory prediction = ",theoryPrediction.upperLimit )

        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = ",r )



 Theory Predictions and Constraints:

 ATLAS-SUSY-2016-07 
------------------------
Dataset =  6j_Meff_1200
TxNames =  ['T1']
Prediction Mass =  [[1.84E+02 [GeV], 1.68E+02 [GeV]], [1.84E+02 [GeV], 1.68E+02 [GeV]]]
Prediction PIDs =  [[[-1000024, -1000022], [1000024, 1000022]], [[-1000024, -1000022], [1000023, 1000022]], [[1000023, 1000022], [1000024, 1000022]], [[-1000024, -1000022], [1000023, 1000022]], [[1000023, 1000022], [1000024, 1000022]], [[-1000024, -1000022], [1000023, 1000022]], [[1000023, 1000022], [1000024, 1000022]]]
Theory Prediction =  1.30E+01 [TeV]:1.96E-05 [pb] (None, None)
Condition Violation =  None
UL for theory prediction =  2.17E+00 [fb]
r =  0.009023237996343697

 ATLAS-SUSY-2016-32 
------------------------
Dataset =  SR1FULL_175
TxNames =  ['THSCPM1b', 'THSCPM2b']
Prediction Mass =  None
Prediction PIDs =  [[[1000022], [1000024]], [[1000022], [-1000024]], [[-1000024], [1000024]], [[1000024, 1000022], [-1000024]], [[-1000024, -1000022], [1000024]], [[1000024, 